In [12]:
# ============================================
# IMPORTACIÓN DE LIBRERÍAS NECESARIAS
# ============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

from imblearn.over_sampling import SMOTE
import joblib  # Para guardar y cargar el modelo
# ============================================
# CARGA DEL DATASET ORIGINAL NO BALANCEADO
# ============================================
dataframe = pd.read_csv('ENFCeliacaSinDatosPerdidos.csv', sep=';')
clasificadores = ['Positivo', 'Negativo']  # 1 = Positivo, 0 = Negativo

FileNotFoundError: [Errno 2] No such file or directory: 'ENFCeliacaSinDatosPerdidos.csv'

In [ ]:
dataframe.head(10)

In [ ]:
dataframe.head(10)

In [ ]:
dataframe.describe()

In [ ]:
dataframe.groupby('DISEASE_DIAGOSE').size()

In [ ]:
import seaborn as sb

#Aseguramos que los numeros son enteros
dataframe['DISEASE_DIAGOSE'] = dataframe['DISEASE_DIAGOSE'].astype(int)
sb.catplot(x='DISEASE_DIAGOSE', hue='DISEASE_DIAGOSE', data=dataframe, kind="count", height = 2.5, aspect=0.8, palette={0:"blue", 1:"red"}, legend = False)
plt.title("Distribucion de Clases")
plt.show()

In [ ]:
# ============================================
# 3. SEPARAR VARIABLES X E Y PARA SMOTE
# ============================================
y = dataframe['DISEASE_DIAGOSE']
X = dataframe.drop('DISEASE_DIAGOSE', axis=1)

print("Distribución antes del balanceo:")
print(y.value_counts())

In [ ]:
# ============================================
# 4. APLICAR SMOTE SOBRE TODO EL DATASET
# ============================================
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Distribución después del balanceo:")
print(y_resampled.value_counts())

In [ ]:
# ============================================
# 5. GUARDAR EL DATASET BALANCEADO EN CSV
# ============================================
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['DISEASE_DIAGOSE'] = y_resampled
df_resampled.to_csv('ENFCeliacaBalanceadoS.csv', index=False, sep=';')

print("✅ Conjunto completo balanceado guardado como 'ENFCeliacaBalanceadoS.csv'")

In [ ]:
# ============================================
# 6. CARGA DEL DATASET BALANCEADO PARA USO
# ============================================
df_balanceado = pd.read_csv('ENFCeliacaBalanceadoS.csv', sep=';')

# Separación de variables
Xb = df_balanceado.drop('DISEASE_DIAGOSE', axis=1)
yb = df_balanceado['DISEASE_DIAGOSE']

In [ ]:
df_balanceado.groupby('DISEASE_DIAGOSE').size()

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

# Asegurar que la columna es entera
df_balanceado['DISEASE_DIAGOSE'] = df_balanceado['DISEASE_DIAGOSE'].astype(int)

# Gráfico de distribución de clases con el dataset balanceado
sb.catplot(
    x='DISEASE_DIAGOSE',
    hue='DISEASE_DIAGOSE',
    data=df_balanceado,
    kind="count",
    height=2.5,
    aspect=0.8,
    palette={0: "blue", 1: "red"},
    legend=False
)
plt.title("Distribución de Clases (Balanceado)")
plt.show()

In [ ]:
# División del dataset balanceado
x_train, x_test, y_train, y_test = train_test_split(Xb, yb, test_size=0.3, random_state=30)

# Entrenar el modelo
knn_mm = KNeighborsClassifier(n_neighbors=6)
knn_mm.fit(x_train, y_train)

# Guardar el modelo entrenado
joblib.dump(knn_mm, 'modelo_entrenado.pkl')
print("✅ Modelo guardado como 'modelo_entrenado.pkl'")

# ============================================================
# 8. CARGA DEL MODELO ENTRENADO
# ============================================================
modelo_cargado = joblib.load('modelo_entrenado.pkl')
print("✅ Modelo cargado desde 'modelo_entrenado.pkl'")

# Predicciones
predicciones = modelo_cargado.predict(x_test)
# Matriz de Confusión
tn, fp, fn, tp = confusion_matrix(y_test, predicciones).ravel()
sb.heatmap(confusion_matrix(y_test, predicciones), annot=True, fmt="d")
plt.title('MATRIZ DE CONFUSIÓN')
plt.xlabel('Predicción')
plt.ylabel('Real')

# Reporte de clasificación
print(classification_report(y_test, predicciones, target_names=clasificadores))

In [ ]:
# ============================================================
# 8. Tasa de error usando dataset balanceado
# ============================================================
tasa_error = []
valores_k = range(1, 27)

for i in valores_k:
    knn_mm = KNeighborsClassifier(n_neighbors=i)
    knn_mm.fit(x_train, y_train)
    prediction_i = knn_mm.predict(x_test)
    tasa_error.append(np.mean(prediction_i != y_test))

# Mostrar la gráfica
plt.plot(valores_k, tasa_error, color="green", marker="o", markerfacecolor="red", markersize=8)
plt.title("Tasa de error vs. número de vecinos (K)")
plt.xlabel("Número de vecinos (K)")
plt.ylabel("Tasa de error")
plt.grid(True)
plt.show()

# Mostrar la tabla con los valores exactos
tabla_resultados = pd.DataFrame({
    'K (número de vecinos)': list(valores_k),
    'Tasa de error': tasa_error
})

display(tabla_resultados)

In [ ]:
knn_mm = KNeighborsClassifier(n_neighbors=1)

#Entrenar el algoritmo
knn_mm.fit(x_train,y_train)

predicciones = knn_mm.predict(x_test)

sb.heatmap(confusion_matrix(y_test,predicciones), annot = True, fmt = "d")
plt.title('MATRIZ DE CONFUSIÓN')
plt.xlabel('Predicción')
plt.ylabel('Real')

print(classification_report(y_test,predicciones,target_names=clasificadores))

In [ ]:
labels = ['Si_tiene', 'No_tiene']
matriz = confusion_matrix(y_test, predicciones, labels=[0, 1])
print(pd.DataFrame(matriz, index=labels, columns=labels))

print(classification_report(y_test, predicciones, target_names=clasificadores))

In [ ]:
knn_mm = KNeighborsClassifier(n_neighbors=2)

#Entrenar el algoritmo
knn_mm.fit(x_train,y_train)

predicciones = knn_mm.predict(x_test)

sb.heatmap(confusion_matrix(y_test,predicciones), annot = True, fmt = "d")
plt.title('MATRIZ DE CONFUSIÓN')
plt.xlabel('Predicción')
plt.ylabel('Real')

print(classification_report(y_test,predicciones,target_names=clasificadores))

In [ ]:
# Conjunto de entrada sin nombres de columna: funciona, pero puede lanzar advertencia
print(knn_mm.predict([[9, 0, 1, 1, 0, 0, 0, 0, 1.50, 12.5, 1.30]]))
print(knn_mm.predict_proba([[9, 0, 1, 1, 0, 0, 0, 0, 1.50, 12.5, 1.30]]))

In [ ]:
print(knn_mm.predict(pd.DataFrame([[15, 0, 0, 0, 0, 0, 0, 0, 0.02,2, 0.20]], columns=Xb.columns)))
print(knn_mm.predict_proba(pd.DataFrame([[15, 0, 0, 0, 0, 0, 0, 0, 0.02, 2, 0.20]], columns=Xb.columns)))